# Imports

In [ ]:
import os
from tqdm.auto import tqdm

from random import seed as rseed

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Pre process
from sklearn.preprocessing import StandardScaler

# Models
from sklearn.linear_model import LinearRegression, Ridge, SGDRegressor, Lasso, Lars, ElasticNet
from sklearn.inspection import permutation_importance
from sklearn.svm import LinearSVR

from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from scipy.optimize import curve_fit

# Train and test
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

# Metrics
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

# Load data

In [ ]:
df = pd.read_csv('data/imputed_pre_and_manifest.csv')
df.shape

<b> Number of visits of patients which were pre-manifest and became manifest </b>

In [ ]:
pre = df['subjid'].isin(df.loc[df['hdcat'] == 2, 'subjid'].unique())
man = df['subjid'].isin(df.loc[df['hdcat'] == 3, 'subjid'].unique())
sel = df[(pre) & (man)]
sel.shape

In [ ]:
x, y = np.unique(sel[sel['hdcat'] == 2].groupby('subjid').subjid.count().values, return_counts=True)
plt.bar(x,y, width=.6)
plt.ylabel('Count')
plt.xticks(range(1, 10))
plt.xlabel('Visits before patients are manifest')

In [ ]:
x, y = np.unique(sel[sel['hdcat'] == 3].groupby('subjid').subjid.count().values, return_counts=True)
plt.bar(x,y, width=.6)
plt.ylabel('Count')
plt.xticks(range(1, 7))
plt.xlabel('Visits when patients are manifest')

# Select training data (only diagnosed AAO, no estimates)

In [ ]:
unknown = df[df.hddiagn_est == 1].groupby('subjid').head(1) # Estimated or unknown
known = df[df.hddiagn_est == 0].groupby('subjid').head(1) # Real HDdiagn

print(unknown.shape, known.shape)

In [ ]:
x, y = np.unique(known['caghigh'], return_counts=True)
plt.bar(x, y)
plt.xlabel('Larger CAG repeat size')
plt.ylabel('Counts')
plt.savefig('figures/cag_counts.pdf', dpi=1200)

<b> AAO duration to AAO </b>

In [ ]:
# Convert
known.loc[known.hddiagn > 0, 'hddiagn'] = (known.age - known.hddiagn)
known.loc[known.hddiagn < 0, 'hddiagn'] = (known.age - known.hddiagn) + 1


fig, axs = plt.subplots(1, 2, figsize=(10,5))
axs[0].boxplot(known.hddiagn)
axs[0].set_ylabel('AAO')
axs[0].set_xticks([])

axs[1].boxplot(known.caghigh)
axs[1].set_ylabel('CAG repeat size')
axs[1].set_xticks([])
pass

# Define and refit Langbehn formula

In [ ]:
def func(x, a, b, c):
    return a + np.exp(c - (b * x))

def refit_langbehn(subset):
    a_langbehn = 21.54
    b_langbehn = 0.1460
    c_langbehn = 9.556
    
    x = subset['caghigh'].values
    y = subset['hddiagn'].values

    popt, pcov = curve_fit(func, x, y, p0=np.array([a_langbehn, b_langbehn, c_langbehn]))
    print('a={:.3f}\nb={:.3f}\nc={:.3f}'.format(*popt))
    return lambda x: func(x, *popt), popt

<b> Original Langbehn formula </b>

In [ ]:
# langbehn_err = lambda x: np.sqrt(35.55 + np.exp(17.72 - 0.3269 * x))
langbehn = lambda x: 21.54 + np.exp( 9.556 - (0.1460 * x))

<b> Langbehn refitted (41<=CAGhigh<=56)

In [ ]:
original_cag_refitted, _ = refit_langbehn(known[(known['caghigh'] >=41) & (known['caghigh'] <= 56)].copy())

<b> Langbehn refitted (full CAG range) </b>

In [ ]:
full_cag_refitted, _ = refit_langbehn(known.copy())

<b> Aggregted Model Langbehn refitted on CAG>=41, linear regression for CAG<=40</b>

In [ ]:
agg_langbehn, agg_popt = refit_langbehn(known[(known['caghigh'] >=41)].copy())

# Linear regression
filt = known[known['caghigh'] <= 40].copy()

x = filt['caghigh'].values.reshape(-1,1)
y = filt['hddiagn'].values

agg_linr = LinearRegression()
agg_linr.fit(x, y)

print('\nLinear Regression:\ncoef={:.3f}\nintercept={:.3f}'.format(*agg_linr.coef_, agg_linr.intercept_))

agg_model = lambda x: np.array([func(xi, *agg_popt) if xi > 40 
                                else agg_linr.intercept_ + (agg_linr.coef_ * xi) for xi in x]).astype(float)

# Train ML and Evaluate all models

In [ ]:
def evaluate(y_true, y_pred, outlier=False):
    # max_ae = np.max(np.abs(y_true - y_pred))
    if outlier:
        u, o = np.mean(y_true), np.std(y_true)
        mask = np.logical_or((y_true < u-o), (y_true > u+o))
        y_true, y_pred = y_true[mask], y_pred[mask]
    
    mae = np.mean(np.abs(y_true - y_pred))
    rmse = np.sqrt(np.mean(np.power(y_true - y_pred, 2)))
    r2 = r2_score(y_true, y_pred)
    return [mae, rmse, r2]

In [ ]:
# The models are trained on static variables which are the same for manifest and pre-manifest patients, namely
# the larger and lower CAG repeatsize, gender, parent AAO, whether the mom and or dad had HD and whether 
# there was a family history for HD.
def train(model, cag_range, fit, cols):
    # Define seeds
    seed_value = 42

    # 1. Set `PYTHONHASHSEED` environment variable at a fixed value
    os.environ['PYTHONHASHSEED'] = str(seed_value)

    # 2. Set `python` built-in pseudo-random generator at a fixed value
    rseed(seed_value)

    # 3. Set `numpy` pseudo-random generator at a fixed value
    np.random.seed(seed_value)

    # Select data
    label = 'hddiagn'
    subset = known.groupby('subjid').first().loc[:, [label] + cols]
    subset = subset.loc[((subset['caghigh'] >= cag_range[0]) & (subset['caghigh'] <= cag_range[1]))]
    print('Dropped {} samples'.format(subset.isnull().any(axis=1).sum()))
    subset.dropna(inplace=True)
    n = subset.shape[0]
    print('{} samples left'.format(n)) 
    
    # Get input and labels
    input_data = subset[cols].values
    targets = subset[label].values
    
    # Evaluation results
    test_results = []
    
    # Fold labels and predictions
    test_labels = []
    test_predictions = []
    
    # Train and Evaluate
    kf = KFold(n_splits=10, shuffle=True, random_state=seed_value)
    for fold, (train_idx, test_idx) in enumerate(kf.split(input_data, targets)):
        # K-fold
        X_train, X_valid = input_data[train_idx], input_data[test_idx]
        y_train, y_valid = targets[train_idx], targets[test_idx]

        # Scale
        if fit:
            model.fit(X_train, y_train)
        
        # prediction
        try:
            pred = model.predict(X_valid)
        except:
            pred = model(X_valid.reshape(-1))
        
        # Evaluate
        test_results.append(evaluate(y_valid, pred))
        
    return pd.DataFrame(np.mean(test_results, axis=0).reshape(1, -1), columns=['MAE', 'RMSE', 'R2'])           

In [ ]:
from sklearn.svm import LinearSVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor



def train_models(cag_min, cag_max):
    langbehn_models = [langbehn, original_cag_refitted, full_cag_refitted, agg_model]
    langbehn_names = ['Langbehn original (41-56)', 'Langbehn refitted (41-56)',
                      'Langbehn refitted (36-59)', 'Linear Regression (36-40) + Langbehn refitted (41-59)'
                     ]
    
    models = langbehn_models + [LinearRegression(),
                                RandomForestRegressor(random_state=42),
                                CatBoostRegressor(verbose=0, random_seed=42),
                                XGBRegressor(random_state=42),
                                LGBMRegressor(random_state=42),
                                MLPRegressor(max_iter=1000, random_state=42),
                                LinearSVR(random_state=42, max_iter=25000),
                                KNeighborsRegressor(5, weights='distance'),
                               ]
    
    names = langbehn_names + ['Linear Regression ({}-{})'.format(cag_min, cag_max),
                              'Random Forest ({}-{})'.format(cag_min, cag_max),
                              'CatBoost ({}-{})'.format(cag_min, cag_max),
                              'XGBoost ({}-{})'.format(cag_min, cag_max),
                              'LGBM ({}-{})'.format(cag_min, cag_max),
                              'MLP ({}-{})'.format(cag_min, cag_max),
                              'Linear SVM ({}-{})'.format(cag_min, cag_max),
                              'KNN ({}-{})'.format(cag_min, cag_max),
                             ]
    all_results = []
    
    # Eval
    for name, mod in zip(names, models):
        print('-' * 40)
        print(name)
        if 'Langbehn' in name:
            results = train(mod, cag_range=(cag_min, cag_max), fit=False, cols=['caghigh'])
        else:
            results = train(mod, cag_range=(cag_min, cag_max), fit=True,
                                              cols=['caghigh', 'caglow', 'sex', 'parentagesx', 'momhd_0.0', 'momhd_1.0',
                                                    'dadhd_0.0', 'dadhd_1.0', 'fhx'])
        results.index = [name]
        all_results.append(list(results.reset_index().values.reshape(-1)))
        print()
    
    # Save
    summary = pd.DataFrame(all_results, columns=['Model', 'MAE', 'RMSE', 'R2']).set_index('Model').sort_values('R2')
    summary.to_csv(os.path.join('tables', 'summary_AAO_{}-{}_models.csv'.format(cag_min, cag_max)), float_format='%.3f')
    return summary

<b> Evaluate/fit models on CAG=41-56 </b>

In [ ]:
cag_min = 41
cag_max = 56

train_models(cag_min, cag_max)

<b> Evaluate/fit models on CAG=36-59 </b>

In [ ]:
cag_min = 36
cag_max = 59

train_models(cag_min, cag_max)